# Heart Disease Veri Setinin Ön İşleme Adımları ve Decision Tree Sınıflandırılması

*Bu veri setini ön işleme adımlarından geçirdikten sonra Decision tree kütüphanesiyle sınıflandırdıktan sonra çeşitli parametrelerle accuracy(doğruluk oranı) nasıl değiştiğini gözlemleyeceğiz.*

**VERİ SETİ HAKKINDA BİLGİLER**


*Kalp hastalığı için oluşturulan veri setinde 500 sample(observation, örnek,satır) 6 feature(attributes,etiket,özellik,label,sütun) bulunmaktadır. Veri seti incelendiğinde son sütunun tahmin etmek istediğimiz özellik olduğunu görebiliriz. Son sütun yani HeartDisease; target(hedef) değişkenimizdir.  

Yapılması gereken;
* Eksik veri olup olmadığını kontrol etmek(varsa tamalamak veya çıkartmak),
* Tekrarlayan örnekler varsa çıkartmak,
* Sınıf dağılımını kontrol ederek veri setinin dengesiz olup olmadığını tespit etmek(dengesizlik varsa SMOTE veya çeşitli örnek artırma-azaltma metodlarını denemek)
* Veri setindeki sütunların istatiksel değerlerini kontrol etmek(aykırı veri olup olmadığını görmek için)



In [1]:
#Veri ön işleme için kullanacağımız kütüphaneler ve veri setinin yüklenmesi
import pandas as pd
import numpy as np

df=pd.read_csv("/kaggle/input/heart-prediction-dataset-quantum/Heart Prediction Quantum Dataset.csv")
df

,Age,Gender,BloodPressure,Cholesterol,HeartRate,QuantumPatternFeature,HeartDisease
0,68,1,105,191,107,8.362241,1
1,58,0,97,249,89,9.249002,0
2,44,0,93,190,82,7.942542,1
3,72,1,93,183,101,6.495155,1
4,37,0,145,166,103,7.653900,1
...,...,...,...,...,...,...,...
495,34,0,126,292,116,9.303403,0
496,41,0,164,248,114,9.067889,0
497,45,1,159,175,75,8.718708,0
498,55,0,107,157,101,7.337650,1


In [2]:
#Eksik verinin olup olmadığının kontrol edilmesi
df.isnull().sum()

Age                      0
Gender                   0
BloodPressure            0
Cholesterol              0
HeartRate                0
QuantumPatternFeature    0
HeartDisease             0
dtype: int64

In [5]:
#Tekrarlayan satırlar kaldırılarak inplace parametresiyle veri setini güncelledik
df.drop_duplicates(inplace=True)

In [6]:
#df'i tekrar yükleyerek herhangi bir tekrarlayan veri olmadığını öğreniyoruz.
df

,Age,Gender,BloodPressure,Cholesterol,HeartRate,QuantumPatternFeature,HeartDisease
0,68,1,105,191,107,8.362241,1
1,58,0,97,249,89,9.249002,0
2,44,0,93,190,82,7.942542,1
3,72,1,93,183,101,6.495155,1
4,37,0,145,166,103,7.653900,1
...,...,...,...,...,...,...,...
495,34,0,126,292,116,9.303403,0
496,41,0,164,248,114,9.067889,0
497,45,1,159,175,75,8.718708,0
498,55,0,107,157,101,7.337650,1


In [14]:
#Tahmin etmek istediğimiz sütunu bir değişkene atıyoruz
target=df["HeartDisease"]
target

0      1
1      0
2      1
3      1
4      1
      ..
495    0
496    0
497    0
498    1
499    0
Name: HeartDisease, Length: 500, dtype: int64

In [16]:
#Sınıfları içeren sütun hariç geri kalanı başka bir veri setinde tutuyoruz.
df=df.drop(columns="HeartDisease", axis=0)
df

,Age,Gender,BloodPressure,Cholesterol,HeartRate,QuantumPatternFeature
0,68,1,105,191,107,8.362241
1,58,0,97,249,89,9.249002
2,44,0,93,190,82,7.942542
3,72,1,93,183,101,6.495155
4,37,0,145,166,103,7.653900
...,...,...,...,...,...,...
495,34,0,126,292,116,9.303403
496,41,0,164,248,114,9.067889
497,45,1,159,175,75,8.718708
498,55,0,107,157,101,7.337650


In [107]:
#Verilerin nasıl değiştiğini kontrol ediyoruz
df.describe()

,Age,Gender,BloodPressure,Cholesterol,HeartRate,QuantumPatternFeature
count,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000
mean,54.864000,0.468000,132.874000,221.50000,88.766000,8.317407
std,14.315004,0.499475,26.418516,43.86363,17.417289,0.919629
min,30.000000,0.000000,90.000000,150.00000,60.000000,6.164692
25%,43.000000,0.000000,111.000000,183.75000,73.000000,7.675779
50%,55.000000,0.000000,132.000000,221.00000,89.000000,8.323064
75%,66.250000,1.000000,155.000000,258.00000,104.000000,8.935999
max,79.000000,1.000000,179.000000,299.00000,119.000000,10.784886


In [15]:
"""Veri setinde dengesizlik olup olmadığını kontrol ediyoruz. Çünkü dengesizlik modelin belirli bir 
sınıfı daha iyi öğrendiği anlamına gelir. Aşağıdaki yorum satırını kaldırarak diğer sınıftan kaç tane 
olduğunu görebilirsiniz. 0=>200 tane 1=>300 tane var. Dengesizlik var. Bunu önlemek için SMOTE 
kullanacağız.
"""

#df[df["HeartDisease"]==0].count()
df[df["HeartDisease"]==1].count()

Age                      300
Gender                   300
BloodPressure            300
Cholesterol              300
HeartRate                300
QuantumPatternFeature    300
HeartDisease             300
dtype: int64

**Model Oluşturulması Ve Eğitim Süreci**

In [17]:
#Needed libs
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

#Veri setini eğitim(%80) ve test(%20) verisi olarak ayır 
X_test, X_train, y_test, y_train = train_test_split(df, target, test_size = 0.2, random_state=99)

#Azınlıkta olan verinin miktarını artırmayı hedefliyoruz
#Smote sadece train kısmına uygulanır teste uygulanmaması gerekir, yoksa aşırı öğrenme gerçekleşir
smote=SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train,y_train)

In [33]:
#Veri artırmanın sonuçlanıp sonuçlanmadığına bakıyoruz
from collections import Counter
print(Counter(y_train_resampled))
print(X_train_resampled.info())

Counter({1: 59, 0: 59})
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Age                    118 non-null    int64  
 1   Gender                 118 non-null    int64  
 2   BloodPressure          118 non-null    int64  
 3   Cholesterol            118 non-null    int64  
 4   HeartRate              118 non-null    int64  
 5   QuantumPatternFeature  118 non-null    float64
dtypes: float64(1), int64(5)
memory usage: 5.7 KB
None


In [34]:
#Modeli başlattık
decisiontree = DecisionTreeClassifier(random_state=42)

#Model eğitimine smote ile değiştirdiğimiz verileri veriyoruz
decisiontree.fit(X_train_resampled, y_train_resampled)

#Test verisiyle tahmin yapılıyor
y_pred = decisiontree.predict(X_test)

#Tahminlerin ne kadar doğru olduğu accuracy(doğruluk oranı) ile hesaplandı
acc= accuracy_score(y_pred, y_test)
print(acc)

0.9025


**Farklı Parametreler İle Değişim**

In [45]:
#Veri setini eğitim(%70) ve test(%30) verisi olarak ayır 
X_test, X_train, y_test, y_train = train_test_split(df, target, test_size = 0.3, random_state=99)

#default olarak çalışan gini yerine entropy kullandık
decisiontree = DecisionTreeClassifier(random_state=42,criterion="entropy")
decisiontree.fit(X_train_resampled, y_train_resampled)
y_pred = decisiontree.predict(X_test)

doğruluk= accuracy_score(y_pred, y_test)
print(doğruluk)

0.8971428571428571


In [47]:
#Veri setini eğitim(%70) ve test(%30) verisi olarak ayır, random_state 99'dan 42'ye çekildi 
X_test, X_train, y_test, y_train = train_test_split(df, target, test_size = 0.3, random_state=42)


#Modeli başlattık
decisiontree = DecisionTreeClassifier(random_state=42,criterion="entropy")

#Model eğitimine smote ile değiştirdiğimiz verileri veriyoruz
decisiontree.fit(X_train_resampled, y_train_resampled)

#Test verisiyle tahmin yapılıyor
y_pred = decisiontree.predict(X_test)

#Tahminlerin ne kadar doğru olduğu accuracy(doğruluk oranı) ile hesaplandı
accur= accuracy_score(y_pred, y_test)
print(accur)

0.9314285714285714


In [50]:
#Veri setini eğitim(%80) ve test(%20) verisi olarak ayır, random_state 42'den 99'a geri getirdik 
X_test, X_train, y_test, y_train = train_test_split(df, target, test_size = 0.3, random_state=99)

#Modeli başlattık
decisiontree = DecisionTreeClassifier(random_state=42,criterion="entropy")

#Model eğitimine smote ile değiştirdiğimiz verileri veriyoruz
decisiontree.fit(X_train_resampled, y_train_resampled)

#Test verisiyle tahmin yapılıyor
y_pred = decisiontree.predict(X_test)

#Tahminlerin ne kadar doğru olduğu accuracy(doğruluk oranı) ile hesaplandı
accuracy= accuracy_score(y_pred, y_test)
print(accuracy)

0.8971428571428571


**SONUÇ**


* Criterion='gini' ve random_state=99;    
* Veri setinin bölünme yüzdesi 0.2 iken doğruluk oranı fazla
* Criterion='entropy' ve random_state=42;
* Veri setinin bölünme yüzdesi 0,3 iken accuracy fazla  Yorumlarını yapabiliriz.  Diğer parametreleri  *DecisionTreeClassifier?* yazarak sizlerde deneyebilirsiniz*


In [42]:
DecisionTreeClassifier?

Init signature:
DecisionTreeClassifier(
    *,
    criterion='gini',
    splitter='best',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0,
)
Docstring:     
A decision tree classifier.

Read more in the :ref:`User Guide <tree>`.

Parameters
----------
criterion : {"gini", "entropy", "log_loss"}, default="gini"
    The function to measure the quality of a split. Supported criteria are
    "gini" for the Gini impurity and "log_loss" and "entropy" both for the
    Shannon information gain, see :ref:`tree_mathematical_formulation`.

splitter : {"best", "random"}, default="best"
    The strategy used to choose the split at each node. Supported
    strategies are "best" to choose the best split and "random" to choose
    the best random split.

max_depth : int, default=None
    The maximum depth o